In [382]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

features_names = [
    'cement', 'water', 'superplasticizer', 'age'   
]
target_name = 'concrete_compressive_strength'
# Load the data
data = pd.read_excel('concrete_data.xlsx')

print(data.head())

# Print the headers (column names)
print("Headers:", data.columns.tolist())

print("Data shape:", data.shape)


   cement  water  superplasticizer  age  concrete_compressive_strength
0   540.0  162.0               2.5   28                          79.99
1   540.0  162.0               2.5   28                          61.89
2   332.5  228.0               0.0  270                          40.27
3   332.5  228.0               0.0  365                          41.05
4   198.6  192.0               0.0  360                          44.30
Headers: ['cement', 'water', 'superplasticizer', 'age', 'concrete_compressive_strength']
Data shape: (700, 5)


In [383]:
x = data[features_names]
y = data[target_name]

scaller = MinMaxScaler()

def normalize(data,columns, is_test=False):
    if is_test:
        data = scaller.transform(data)
    else:
        data = scaller.fit_transform(data)
    return pd.DataFrame(data, columns=columns)
      
   
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, shuffle=True)
x_train = normalize(x_train, features_names)
x_test = normalize(x_test, is_test=True, columns=features_names)

scaller2 = MinMaxScaler()
y_train = scaller2.fit_transform(y_train.values.reshape(-1, 1))
y_test = scaller2.transform(y_test.values.reshape(-1, 1))
# y_train = normalize(y_train.values.reshape(-1, 1), columns=[target_name])
# y_test = normalize(y_test.values.reshape(-1, 1), is_test=True, columns=[target_name])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(525, 4) (175, 4) (525, 1) (175, 1)


In [384]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size,epochs=10000, learning_rate=0.5):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.previos_error = 0
        self.init_weights()

    def init_weights(self):
        self.weights1 = np.random.uniform(-1, 1, (self.input_size, self.hidden_size))
        self.weights2 = np.random.uniform(-1, 1, (self.hidden_size, self.output_size))
        self.hidden_layer_bias = np.random.uniform(-1, 1, (1, self.hidden_size))
        self.output_layer_bias = np.random.uniform(-1, 1, (1, self.output_size))
        # self.weights1 = np.full((self.input_size, self.hidden_size), 0.5)
        # self.weights2 = np.full((self.hidden_size, self.output_size), 0.3)
        # self.hidden_layer_bias = np.full((1, self.hidden_size), 0.4)
        # self.output_layer_bias = np.full((1, self.output_size), 0.2)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, x):
        hidden_layer_input = np.dot(x, self.weights1) + self.hidden_layer_bias
        hidden_layer_output = self.sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_output, self.weights2) + self.output_layer_bias
        output_layer_output = self.sigmoid(output_layer_input)
        return output_layer_output, hidden_layer_output

    def backward(self, x, y, output, hidden_layer_output):
        
        # Step 1: Calculate output error and delta
        output_error = y - output
        output_layer_delta = self.sigmoid_derivative(output) * output_error

        # Step 2: Update output layer weights and biases
        self.weights2 += self.learning_rate * np.dot(hidden_layer_output.T, output_layer_delta)
        # self.output_layer_bias += self.learning_rate * np.sum(output_layer_delta, axis=0, keepdims=True)

        # Step 3: Backpropagate error to hidden layer
        hidden_layer_error = np.dot(output_layer_delta, self.weights2.T)
        hidden_layer_delta = self.sigmoid_derivative(hidden_layer_output) * hidden_layer_error

        # Step 4: Update hidden layer weights and biases
        self.weights1 += self.learning_rate * np.dot(x.T, hidden_layer_delta)
        # self.hidden_layer_bias += self.learning_rate * np.sum(hidden_layer_delta, axis=0, keepdims=True)

    def train(self, x, y, patience=11):
        
        epochs_without_improvement = 0
        best_error = float('inf')
        # Ensure input data is a numpy array
        x = np.array(x)
        y = np.array(y).reshape(-1, 1)  # Ensure y is column vector
        for i in range(self.epochs):
            # Forward pass
            final_output, hidden_layer_output = self.forward(x)

            # Backward pass
            self.backward(x, y, final_output, hidden_layer_output)

            # Calculate error
            error = np.mean(0.5 * ((y - final_output) ** 2))
            # print(f"Epoch {i+1}, Error: {error}")
            
            if error < best_error:
                print(f"Error improved from {best_error} to {error}.")
                best_error = error
                epochs_without_improvement = 0
            else:
                print(f"Error has not improved from {best_error}.")
                epochs_without_improvement += 1
                if epochs_without_improvement == patience:
                    print(f"Training stopped as error has not improved in {patience} epochs.")
                    break

            # # Early stopping
            # if error <= 0.001:
            #     print("Training stopped as error is below threshold.")
            #     break

    def predict(self, x):
        x = np.array(x)
        return self.forward(x)[0]


# Example Usage
nn = NeuralNetwork(input_size=4, hidden_size=8, output_size=1, epochs=10000, learning_rate = 0.01) # 0.07 0.05 0.1 5000
"8 15000 0.01  => 0.07"
# Train the network
nn.train(x_train, y_train)

# Predict and evaluate
predictions = nn.predict(x_test)
mse_test = np.mean(0.5 * ((y_test - predictions) ** 2))
accuracy = 1 - mse_test
from sklearn.metrics import r2_score


r2 = r2_score(y_test, predictions)



print(f"Test MSE: {mse_test}")
print(f"Test Accuracy: {accuracy}")

print(f"R2 Score: {r2}")


Error improved from inf to 0.02338379424372184.
Error improved from 0.02338379424372184 to 0.02266239743472273.
Error improved from 0.02266239743472273 to 0.022454124460085403.
Error improved from 0.022454124460085403 to 0.022356175167893247.
Error improved from 0.022356175167893247 to 0.02228235677180717.
Error improved from 0.02228235677180717 to 0.022213931214701468.
Error improved from 0.022213931214701468 to 0.022146754809005603.
Error improved from 0.022146754809005603 to 0.022079894822532198.
Error improved from 0.022079894822532198 to 0.022013132608595647.
Error improved from 0.022013132608595647 to 0.02194640980032856.
Error improved from 0.02194640980032856 to 0.02187970469113705.
Error improved from 0.02187970469113705 to 0.021813004502264578.
Error improved from 0.021813004502264578 to 0.02174629911540437.
Error improved from 0.02174629911540437 to 0.02167957963251258.
Error improved from 0.02167957963251258 to 0.02161283802975493.
Error improved from 0.02161283802975493 to